In [15]:
import urllib.request
import os
import pandas as pd
import numpy as np
from datetime import datetime
import glob
from io import StringIO

In [16]:
foldername = "provinces"
os.makedirs(foldername, exist_ok=True)

In [42]:
ukr_provinces_ordered = [
    "Вінницька", "Волинська", "Дніпропетровська", "Донецька", "Житомирська",
    "Закарпатська", "Запорізька", "Івано-Франківська", "Київська", "Кіровоградська",
    "Луганська", "Львівська", "Миколаївська", "Одеська", "Полтавська",
    "Рівненська", "Сумська", "Тернопільська", "Харківська", "Херсонська",
    "Хмельницька", "Черкаська", "Чернівецька", "Чернігівська",
    "м.Київ", "АР Крим"
]


In [18]:
def download_province(province_id: int):
    if province_id == 0:  # пропускаємо середнє по Україні
        return

    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1=1981&year2=2024&type=Mean"
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"Province{province_id}_{timestamp}.csv"
    filepath = os.path.join(foldername, filename)

    existing_files = glob.glob(f"{foldername}/Province{province_id}_*.csv")
    if existing_files:
        print(f"[SKIP] Файл для provinceID={province_id} вже існує ({len(existing_files)} шт.)")
        return

    try:
        print(f"[INFO] Завантаження provinceID={province_id}...")
        urllib.request.urlretrieve(url, filepath)
        print(f"[OK] Збережено: {filepath}")
    except Exception as e:
        print(f"[ERROR] Не вдалося завантажити provinceID={province_id}: {e}")


In [32]:
for pid in range(1, 28):
    download_province(pid)

[SKIP] Файл для provinceID=1 вже існує (17 шт.)
[SKIP] Файл для provinceID=2 вже існує (15 шт.)
[SKIP] Файл для provinceID=3 вже існує (15 шт.)
[SKIP] Файл для provinceID=4 вже існує (15 шт.)
[SKIP] Файл для provinceID=5 вже існує (14 шт.)
[SKIP] Файл для provinceID=6 вже існує (14 шт.)
[SKIP] Файл для provinceID=7 вже існує (14 шт.)
[SKIP] Файл для provinceID=8 вже існує (14 шт.)
[SKIP] Файл для provinceID=9 вже існує (14 шт.)
[SKIP] Файл для provinceID=10 вже існує (14 шт.)
[SKIP] Файл для provinceID=11 вже існує (14 шт.)
[SKIP] Файл для provinceID=12 вже існує (14 шт.)
[SKIP] Файл для provinceID=13 вже існує (14 шт.)
[SKIP] Файл для provinceID=14 вже існує (14 шт.)
[SKIP] Файл для provinceID=15 вже існує (14 шт.)
[SKIP] Файл для provinceID=16 вже існує (14 шт.)
[SKIP] Файл для provinceID=17 вже існує (14 шт.)
[SKIP] Файл для provinceID=18 вже існує (14 шт.)
[SKIP] Файл для provinceID=19 вже існує (14 шт.)
[SKIP] Файл для provinceID=20 вже існує (14 шт.)
[SKIP] Файл для provinceID=21

In [33]:
all_provinces_data = []

for pid in range(1, 28):
    files = glob.glob(f"{foldername}/Province{pid}_*.csv")
    if not files:
        print(f"[WARNING] Немає файлів для provinceID={pid}")
        continue

    latestfile = sorted(files)[-1]
    print(f"\n=== Область {pid}: файл {latestfile} ===")

    with open(latestfile, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Видаляємо HTML-теги та зайві рядки
    lines = [line.replace('<tt><pre>', '').replace('</pre></tt>', '') for line in lines]
    data_lines = [line for line in lines if not line.startswith('Mean') and not line.startswith('year')]
    data_lines = [line.rstrip().rstrip(',') + '\n' for line in data_lines]

    data_csv = ''.join(data_lines)
    columns = ['year', 'week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI']
    df = pd.read_csv(StringIO(data_csv), names=columns)

    # Замінюємо -1 та всі від'ємні значення на NaN
    for col in ['SMN','SMT','VCI','TCI','VHI']:
        df[col] = df[col].apply(lambda x: np.nan if x <= 0 else x)

    # Заповнюємо пропуски: спочатку forward fill, потім backward fill
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)

    # Додаємо колонку з англійським порядком (тимчасово)
    df['old_province_id'] = pid

    all_provinces_data.append(df)

# Об'єднуємо всі області в один DataFrame
df_all = pd.concat(all_provinces_data, ignore_index=True)

# Перенумерація областей за українською абеткою
old_to_new_id = {old_id: new_id+1 for new_id, old_id in enumerate(range(1,28))}
df_all['province_id'] = df_all['old_province_id'].map(old_to_new_id)
df_all['province_name'] = df_all['province_id'].map({i+1: name for i, name in enumerate(ukr_provinces_ordered)})

# Видаляємо тимчасову колонку
df_all.drop(columns=['old_province_id'], inplace=True)

# Вивід для перевірки
print("\n=== Перші 5 рядків об'єднаного та очищеного DataFrame ===")
print(df_all.head())

print("\n=== Загальна статистика по всіх областях ===")
print(df_all.describe())



=== Область 1: файл provinces\Province1_20250929_10h49m51s.csv ===

=== Область 2: файл provinces\Province2_20250929_10h49m51s.csv ===

=== Область 3: файл provinces\Province3_20250929_10h49m52s.csv ===

=== Область 4: файл provinces\Province4_20250929_10h49m53s.csv ===

=== Область 5: файл provinces\Province5_20250929_10h49m54s.csv ===

=== Область 6: файл provinces\Province6_20250929_10h49m55s.csv ===

=== Область 7: файл provinces\Province7_20250929_10h49m56s.csv ===

=== Область 8: файл provinces\Province8_20250929_10h49m57s.csv ===

=== Область 9: файл provinces\Province9_20250929_10h49m57s.csv ===

=== Область 10: файл provinces\Province10_20250929_10h49m58s.csv ===

=== Область 11: файл provinces\Province11_20250929_10h49m59s.csv ===

=== Область 12: файл provinces\Province12_20250929_10h50m00s.csv ===

=== Область 13: файл provinces\Province13_20250929_10h50m01s.csv ===


C:\Users\Даня\AppData\Local\Temp\ipykernel_7848\200505177.py:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\Даня\AppData\Local\Temp\ipykernel_7848\200505177.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)
C:\Users\Даня\AppData\Local\Temp\ipykernel_7848\200505177.py:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\Даня\AppData\Local\Temp\ipykernel_7848\200505177.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)
C:\Users\Даня\AppData\Local\


=== Область 14: файл provinces\Province14_20250929_10h50m02s.csv ===

=== Область 15: файл provinces\Province15_20250929_10h50m03s.csv ===

=== Область 16: файл provinces\Province16_20250929_10h50m04s.csv ===

=== Область 17: файл provinces\Province17_20250929_10h50m04s.csv ===

=== Область 18: файл provinces\Province18_20250929_10h50m05s.csv ===

=== Область 19: файл provinces\Province19_20250929_10h50m06s.csv ===

=== Область 20: файл provinces\Province20_20250929_10h50m07s.csv ===

=== Область 21: файл provinces\Province21_20250929_10h50m08s.csv ===

=== Область 22: файл provinces\Province22_20250929_10h50m09s.csv ===

=== Область 23: файл provinces\Province23_20250929_10h50m10s.csv ===

=== Область 24: файл provinces\Province24_20250929_10h50m10s.csv ===

=== Область 25: файл provinces\Province25_20250929_10h50m11s.csv ===


C:\Users\Даня\AppData\Local\Temp\ipykernel_7848\200505177.py:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\Даня\AppData\Local\Temp\ipykernel_7848\200505177.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)
C:\Users\Даня\AppData\Local\Temp\ipykernel_7848\200505177.py:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\Даня\AppData\Local\Temp\ipykernel_7848\200505177.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)
C:\Users\Даня\AppData\Local\


=== Область 26: файл provinces\Province26_20250929_10h50m12s.csv ===

=== Область 27: файл provinces\Province27_20250929_10h50m13s.csv ===

=== Перші 5 рядків об'єднаного та очищеного DataFrame ===
   year  week    SMN     SMT    VCI    TCI    VHI  province_id province_name
0  1982     1  0.053  260.31  45.01  39.46  42.23            1     Вінницька
1  1982     2  0.054  262.29  46.83  31.75  39.29            1     Вінницька
2  1982     3  0.055  263.82  48.13  27.24  37.68            1     Вінницька
3  1982     4  0.053  265.33  46.09  23.91  35.00            1     Вінницька
4  1982     5  0.050  265.66  41.46  26.65  34.06            1     Вінницька

=== Загальна статистика по всіх областях ===
               year          week           SMN           SMT           VCI  \
count  60372.000000  60372.000000  60372.000000  60372.000000  60372.000000   
mean    2003.000000     26.500000      0.231702    283.017845     51.367371   
std       12.409776     15.008455      0.142849     14.2

C:\Users\Даня\AppData\Local\Temp\ipykernel_7848\200505177.py:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
C:\Users\Даня\AppData\Local\Temp\ipykernel_7848\200505177.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [34]:

files = glob.glob("provinces/Province23_*.csv")
if files:
    latestfile = sorted(files)[-1]
    print(f"Читаю файл: {latestfile}")

    with open(latestfile, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    lines = [line.replace('<tt><pre>', '').replace('</pre></tt>', '') for line in lines]
    data_lines = [line for line in lines if  not line.startswith('Mean') and not line.startswith('year')]

    data_lines = [line.rstrip().rstrip(',')+ '\n'  for line in data_lines] #якщо закоментувати, буде зайва кома та відповідно зайва колонка

    data_csv = ''.join(data_lines)
    #print(data_csv) #просто як виглядає один рядок

    columns = ['year', 'week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI']
    df = pd.read_csv(StringIO(data_csv), names=columns)

    #df = df.dropna(how='all')

    print(df.head())
    print(df.describe())

Читаю файл: provinces\Province23_20250929_10h50m10s.csv
   year  week    SMN     SMT    VCI    TCI    VHI
0  1982     1  0.061  253.43  17.93  90.95  54.44
1  1982     2  0.059  255.61  16.96  85.31  51.13
2  1982     3  0.060  258.56  16.90  74.56  45.73
3  1982     4  0.060  261.17  16.45  63.52  39.98
4  1982     5  0.061  263.92  15.86  53.34  34.60
             year         week          SMN          SMT          VCI  \
count  2236.00000  2236.000000  2236.000000  2236.000000  2236.000000   
mean   2003.00000    26.500000     0.271732   275.460210    51.681561   
std      12.41245    15.011688     0.252631    43.097792    23.527545   
min    1982.00000     1.000000    -1.000000    -1.000000    -1.000000   
25%    1992.00000    13.750000     0.127000   271.705000    33.010000   
50%    2003.00000    26.500000     0.292500   284.110000    53.855000   
75%    2014.00000    39.250000     0.461000   291.130000    71.132500   
max    2024.00000    52.000000     0.570000   296.940000    

In [43]:
#Ряд VHI для області за вказаний рік
def get_vhi_all_provinces_year(df, year):
    """
    Повертає VHI для всіх областей за вказаний рік.
    """
    df_filtered = df[df['year'] == year]
    if df_filtered.empty:
        print(f"[INFO] Даних за {year} немає")
        return pd.DataFrame()
    print(f"VHI для всіх областей у {year}:")
    return df_filtered[['province_name','year','week','VHI']]

In [44]:
# Ряд VHI за вказаний діапазон років для областей
def get_vhi_all_provinces_years(df, year_start, year_end):
    """
    Повертає VHI для всіх областей за діапазон років.
    """
    df_filtered = df[(df['year'] >= year_start) & (df['year'] <= year_end)]
    if df_filtered.empty:
        print(f"[INFO] Даних за {year_start}-{year_end} немає")
        return pd.DataFrame()
    print(f"VHI для всіх областей у {year_start}-{year_end}:")
    return df_filtered[['province_name','year','week','VHI']]


In [45]:
# Задача: Пошук екстремумів (min та max), середнього, медіани
def vhi_statistics_all_provinces(df, year_start=None, year_end=None):
    """
    Обчислює статистику (min, max, mean, median) для VHI для всіх областей.
    """
    df_filtered = df.copy()
    if year_start is not None:
        df_filtered = df_filtered[df_filtered['year'] >= year_start]
    if year_end is not None:
        df_filtered = df_filtered[df_filtered['year'] <= year_end]
    
    if df_filtered.empty:
        print(f"[INFO] Даних у зазначений період немає")
        return pd.DataFrame()
    
    stats = df_filtered.groupby('province_name')['VHI'].agg(['min','max','mean','median'])
    print(f"Статистика VHI для всіх областей у {year_start}-{year_end}:")
    return stats

In [46]:
# Вивід VHI для області "Вінницька" у 2003 році
vhi_all_2003 = get_vhi_all_provinces_year(df_all, 2003)
print(vhi_all_2003.head())

# VHI для всіх областей за 2000-2005 роки
vhi_all_2000_2005 = get_vhi_all_provinces_years(df_all, 2000, 2005)
print(vhi_all_2000_2005.head())

# Статистика VHI для всіх областей за 2000-2005 роки
vhi_stats_all = vhi_statistics_all_provinces(df_all, 2000, 2005)
print(vhi_stats_all)

VHI для всіх областей у 2003:
     province_name  year  week    VHI
1092     Вінницька  2003     1  49.76
1093     Вінницька  2003     2  45.72
1094     Вінницька  2003     3  44.91
1095     Вінницька  2003     4  45.02
1096     Вінницька  2003     5  43.94
VHI для всіх областей у 2000-2005:
    province_name  year  week    VHI
936     Вінницька  2000     1  35.79
937     Вінницька  2000     2  37.89
938     Вінницька  2000     3  37.46
939     Вінницька  2000     4  36.62
940     Вінницька  2000     5  37.63
Статистика VHI для всіх областей у 2000-2005:
                     min    max       mean  median
province_name                                     
Івано-Франківська   9.36  91.42  54.944327  55.565
АР Крим            15.92  87.30  50.224263  48.955
Волинська          15.17  75.11  52.049808  52.185
Вінницька          10.68  82.09  51.748558  52.315
Дніпропетровська   18.83  69.31  52.206731  53.925
Донецька           22.30  77.60  46.494519  44.650
Житомирська        17.77  90.29

In [41]:
import glob
files = glob.glob("provinces/*.csv")
print(len(files), files[:27])

411 ['provinces\\Province1.csv', 'provinces\\Province10.csv', 'provinces\\Province10_20250928_17h11m17s.csv', 'provinces\\Province10_20250928_17h27m21s.csv', 'provinces\\Province10_20250928_17h28m48s.csv', 'provinces\\Province10_20250928_17h29m53s.csv', 'provinces\\Province10_20250928_17h32m05s.csv', 'provinces\\Province10_20250928_19h30m20s.csv', 'provinces\\Province10_20250928_19h31m13s.csv', 'provinces\\Province10_20250928_19h35m25s.csv', 'provinces\\Province10_20250928_19h36m47s.csv', 'provinces\\Province10_20250928_19h39m49s.csv', 'provinces\\Province10_20250928_19h49m37s.csv', 'provinces\\Province10_20250928_19h52m59s.csv', 'provinces\\Province10_20250929_10h05m03s.csv', 'provinces\\Province10_20250929_10h49m58s.csv', 'provinces\\Province11.csv', 'provinces\\Province11_20250928_17h11m18s.csv', 'provinces\\Province11_20250928_17h27m22s.csv', 'provinces\\Province11_20250928_17h28m49s.csv', 'provinces\\Province11_20250928_17h29m54s.csv', 'provinces\\Province11_20250928_17h32m06s.csv